In [ ]:
# imports

import numpy as np
#import ROOT
import math
import matplotlib.pyplot as plt

In [ ]:
# Some important parameters and constants and what not

# Fermi Constant in natural units
GF = (1.166*(10**(-5)))*(1000**(-2)) #GeV−2 --> MeV^-2

# weak mixing angle contribution --> PDG value for now --> could vary
sin2thetaW = 0.231

# electroweak coupling coefficients of up and down quarks respectfully
guv = 0.5 - (4./3)*sin2thetaW
gdv = -0.5 + (2./3)*sin2thetaW

# muon and pion masses
m_mu = 105.66 # MeV
m_pi = 139.6 # MeV

In [ ]:
# Functions

def gV(N, Z):
    return N*(guv + 2*gdv) + Z*(2*guv + gdv)


# Return the minimum allowed neutrino energy to produce a nuclear recoil with energy Er when scattering off a nucleus of mass M
def get_nu_min_energy(Er, M):
    return np.sqrt(M*Er/2)


# Get the Recoil energy Er from the incident neutrino energy Enu and the recoil angle thetar
def get_Er_kinematic(Enu, costhetar, M):
    num = 2*M*(Enu**2)*(costhetar**2)
    den = (Enu + M)**2 - ((Enu**2)*costhetar**2)
    return num/den


def get_Enu_kinematic(Er, costhetar, M):
    gamma = (Er + M)*(costhetar**2)
    A = Er - gamma
    B = 2*M*Er
    C = Er*(M**2)
    return (-1.*B + np.sqrt(B**2 - 4.*A*C))/(2.*A)


# Get a convenient parameter denoted epsilon that depends on various things
def get_epsilon(Er, costhetar, M):
    Enu_min = get_nu_min_energy(Er, M)
    return 1.0/((costhetar/Enu_min) - (1.0/M))


def dsigma_dEr(Er, costhetar, M, N, Z):
    Enu = get_Enu_kinematic(Er, costhetar, M)
    A = 2 - ((M*Er)/(Enu**2))
    B = (M*GF**2)/(2*math.pi)
    C = gV(N, Z)**2
    return A*B*C


def ddf(x, mu, sig):
    val = np.zeros_like(x)
    val[((mu-sig) <=x) & (x<= (mu +sig))] = 1
    return val

# spherical Bessel Function of the first kind
def j1(q):
    A = np.sin(q)/(q**2)
    B = np.cos(q)/q
    return A - B
    

print("made useful functions")


In [ ]:
x = np.linspace(0, 100, 10000)

y = ddf(x, 30, 0.01)

plt.plot(x, y)
plt.show()

# Make the Spectral Function for the Nu_e Source

In [ ]:
 

def spectral_nue(Enu):
    A = 192./m_mu
    B = (Enu/m_mu)**2
    C = 0.5 - (Enu/m_mu)
    S = (Enu <= (m_mu/2))
    return A*B*C*S

def spectral_nue_arr(Enus):
    maxv = m_mu/2
    s = np.zeros_like(Enus)
    s[(Enus <= maxv)] = 1
    fs = spectral_nue(Enus)
    return s*fs


Es = np.linspace(0, 0.6*m_mu, 1000)
#fs = spectral_nue_arr(Es)
fs = spectral_nue(Es)
plt.plot(Es, fs)
plt.show()

print(spectral_nue(50))

# Make the Spectral Function for the Nu_mu Source

In [ ]:
def spectral_numu(Enu):
    V = (m_pi**2 - m_mu**2)/(2*m_pi)
    dE = 0.001 #MeV
    S1 = (Enu >= V - dE)
    S2 = (Enu <= V + dE)
    return ((2*m_pi)/(m_pi**2 - m_mu**2))*S1*S2

def spectral_numu_arr(Enus):
    dE = (Enus[1] - Enus[0])/2
    s = np.zeros_like(Enus)
    V = (m_pi**2 - m_mu**2)/(2*m_pi)
    s[(Enus <= V + dE) & (Enus >= V - dE)] = 1
    return s*spectral_numu(Enus)


Es = np.linspace(0, 100, 100000)
fs = spectral_numu(Es)

plt.plot(Es, fs)
plt.show()

# Make the Spectral Function for the Nu_mu Bars

In [ ]:
 

def spectral_numubar(Enu):
    A = 64./m_mu
    B = (Enu/m_mu)**2
    C = 0.75 - (Enu/m_mu)
    maxv = m_mu/2
    S = (Enu <= maxv)
    return A*B*C*S

def spectral_numubar_arr(Enus):
    maxv = m_mu/2
    s = np.zeros_like(Enus)
    s[(Enus <= maxv)] = 1
    fs = spectral_numubar(Enus)
    return s*fs


Es = np.linspace(0, 0.8*m_mu, 1000)
fs = spectral_numubar(Es)

plt.plot(Es, fs)
plt.show()

# Show all the Spectral Functions Together

In [ ]:
Es = np.linspace(0, 100, 100000)
f_numu = spectral_numu(Es)
f_nue = spectral_nue(Es)
f_numubar = spectral_numubar(Es)

plt.plot(Es, f_nue, label=r"$\nu_{e}$")
plt.plot(Es, f_numu, label=r"$\nu_{\mu}$")
plt.plot(Es, f_numubar, label=r"$\bar{\nu}_{\mu}$")
plt.xlabel("Neutrino Energy [MeV]", fontsize=14)
plt.ylabel("Relative Spectral Shape [Arb.]", fontsize=14)
plt.ylim([0.0001, 0.04])
plt.xlim([0, 53])
plt.legend()
plt.show()

In [ ]:
print(np.sum(f_nue)*(Es[1] - Es[0]))
print(np.sum(f_numubar)*(Es[1] - Es[0]))
print(np.sum(f_numu)*(Es[1] - Es[0]))

# Normalize the Flux to the SNS Source

In [ ]:
r = 0.08 # neutrinos produced / POT
#L = 29.0 # meters --> distance to the detector
L = 20.0 # meters

Nflux = r / (4*math.pi*(L**2)) # neutrinos / POT / Area

def flux_numu(Enu):
    A = (m_pi**2 - m_mu**2)/(2*m_pi)
    return spectral_numu(Enu)*Nflux*A

def flux_numubar(Enu):
    return spectral_numubar(Enu)*Nflux

def flux_nue(Enu):
    return spectral_nue(Enu)*Nflux

def flux_total(Enu):
    f1 = flux_nue(Enu)
    f2 = flux_numu(Enu)
    f3 = flux_numubar(Enu)
    return f1 + f2 + f3

Es = np.linspace(0, 100, 100000)
#Es = 50
f_numu = flux_numu(Es)
f_nue = flux_nue(Es)
f_numubar = flux_numubar(Es)
f_tot = flux_total(Es)

plt.plot(Es, f_nue, label=r"$\nu_{e}$")
plt.plot(Es, f_numu, label=r"$\nu_{\mu}$")
plt.plot(Es, f_numubar, label=r"$\bar{\nu}_{\mu}$")
plt.plot(Es, f_tot, c="black", label="Total")
plt.xlabel("Neutrino Energy [MeV]", fontsize=14)
plt.ylabel("Neutrino Flux ["+r"$\nu$"+"/POT/"+r"$m^2$"+"]", fontsize=14)
#plt.ylim([0.0001, 0.04])
plt.xlim([0, 52.8])
plt.yscale("log")
plt.legend()
plt.show()

# Test that we can integrate the Flux Numerically Using Quad

In [ ]:
from scipy.integrate import quad

test = quad(flux_numubar, 0, 53)

#test = quad(flux_numu, 0, 53)

#test = quad(spectral_numubar, 0, 53)

print(test[0])
print(Nflux)

# Form Factors

In [ ]:
#hbar = (4.1357*10**-15)/(2*math.pi*10**6) # MeV * seconds 

hbar = 6.582*(10**(-16))*(1.0/(10**6))
c = 3*10**8 # m /second (speed of light of course)

M = 37227.56 # MeV for Argon

Rmin = 3.427*10**-15 # m for Argon? --> Should double check this

# Helm Form Factor
def FH(Er, M, Rmin):
    s = 0.9*10**(-15) # m
    Ro = np.sqrt((5./3)*(Rmin**2 - (3*s**2)))
    q = np.sqrt(2*M*Er)/(hbar*c)
    a = q*Ro
    A = 3*j1(a)/a
    B = -0.5*(q*s)**2
    return A*np.exp(B)

Es = np.linspace(0.001, 0.1, 1000)
qs = np.sqrt(2*M*Es)
fs = FH(Es, M, Rmin)**2
plt.plot(qs, fs)
plt.show()

print(hbar)

# Differential Cross Section

In [ ]:

def dSigma_dEr(Er, Enu, M, N, Z):
    A = (GF**2)*M/(2*math.pi)
    B = 2 - (M*Er/(Enu**2))
    C = gV(N, Z)**2
    D = (hbar*c)**2
    return A*B*C*D


Es = np.linspace(0, 0.1, 1000)
sigmas = dSigma_dEr(Es, 50, M, 22, 18)

plt.plot(Es, sigmas*(100**2))
plt.show()

# Neutrino Cross Section

In [ ]:
def Sigma(Enu, M, N, Z):
    
    def integrand(Er, Enu, M, N, Z):
        return dSigma_dEr(Er, Enu, M, N, Z)*(FH(Er, M, Rmin)**2)
        
    v = []
    for E in Enu:
        m = (2*E**2)/M
        I = quad(integrand, 0, m, args=(E, M, N, Z))
        v.append(I[0])
    return np.array(v)


Enus = np.linspace(0.00001, 53, 1000)
sigmas = Sigma(Enus, M, 22, 18)


plt.plot(Enus, sigmas*(100**2)*(10**38))
plt.xlim([5, 53])
plt.ylim([10**(-5), 10**2])
plt.yscale("log")
plt.ylabel("Cross-section ("+r"$10^{-38} cm^2$"+")", fontsize=14)
plt.xlabel("Neutrino Energy [MeV]", fontsize=14)
plt.show()

# Rate as a Function of Energy

In [ ]:
def dR_dEr_tot(Er, M, N, Z, Norm):
    def integrand(Enu, Er, M, N, Z):
        f = flux_total(Enu)
        Fsquared = FH(Er, M, Rmin)**2
        sig = dSigma_dEr(Er, Enu, M, N, Z)
        return f*Fsquared*sig
    v = []  
    for E in Er:
        a = get_nu_min_energy(E, M)
        b = m_mu/2
        I = quad(integrand, a, b, args=(E, M, N, Z))
        v.append(I[0])
    return Norm*np.array(v)


def dR_dEr_nue(Er, M, N, Z, Norm):
    def integrand(Enu, Er, M, N, Z):
        f = flux_nue(Enu)
        Fsquared = FH(Er, M, Rmin)**2
        sig = dSigma_dEr(Er, Enu, M, N, Z)
        return f*Fsquared*sig
    v = []
    try:
        for E in Er:
            a = get_nu_min_energy(E, M)
            b = m_mu/2
            I = quad(integrand, a, b, args=(E, M, N, Z))
            v.append(I[0])
        return Norm*np.array(v)
        
    except:
        a = get_nu_min_energy(Er, M)
        b = m_mu/2
        I = quad(integrand, a, b, args=(Er, M, N, Z))
        return Norm*I[0]

def dR_dEr_numubar(Er, M, N, Z, Norm):
    def integrand(Enu, Er, M, N, Z):
        f = flux_numubar(Enu)
        Fsquared = FH(Er, M, Rmin)**2
        sig = dSigma_dEr(Er, Enu, M, N, Z)
        return f*Fsquared*sig
    v = []
    try:
        for E in Er:
            a = get_nu_min_energy(E, M)
            b = m_mu/2
            I = quad(integrand, a, b, args=(E, M, N, Z))
            v.append(I[0])
        return Norm*np.array(v)
        
    except:
        a = get_nu_min_energy(Er, M)
        b = m_mu/2
        I = quad(integrand, a, b, args=(Er, M, N, Z))
        return Norm*I[0]

def dR_dEr_numu(Er, M, N, Z, Norm):
    Emono = (m_pi**2 - m_mu**2)/(2*m_pi)
    m = (2*Emono**2)/M
    f = Nflux
    Fsquared = FH(Er, M, Rmin)**2
    sig = dSigma_dEr(Er, Emono, M, N, Z)
    S = (Er <= m) 
    return Norm*f*Fsquared*sig*S

mdet = 1000 # kg
mavg_Ar = 39.947*(1/1000) # kg / mol
NA = 6.022*(10**23) # Avagadro's number [1/mol]
Norm_Ar_COHERENT_ton = mdet*(NA/mavg_Ar)
#Norm_Ar_COHERENT_ton = NA/mavg_Ar

N_POT = (1.76*(10**23))*(365/308.1) # POT in a year

Ers = np.linspace(0.001, 0.14, 10000)


rates0 = dR_dEr_tot(Ers, M, 22, 18, 1)
rate_numu0 = dR_dEr_numu(Ers, M, 22, 18, 1)
rate_numubar0 = dR_dEr_numubar(Ers, M, 22, 18, 1)
rate_nue0 = dR_dEr_nue(Ers, M, 22, 18, 1)
#plt.plot(Ers*1000, rates0, c="black", label="Total")
plt.plot(Ers*1000, rate_nue0, label=r"$\nu_{e}$")
#plt.plot(Ers*1000, rate_numu0, label=r"$\nu_{\mu}$")
plt.plot(Ers*1000, rate_numubar0, label=r"$\bar{\nu}_{\mu}$")
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
#plt.yscale("log")
plt.ylabel("Events/POT/Nt")
plt.legend()
plt.show()



rates = dR_dEr_tot(Ers, M, 22, 18, N_POT*Norm_Ar_COHERENT_ton)
rate_numu = dR_dEr_numu(Ers, M, 22, 18, N_POT*Norm_Ar_COHERENT_ton)
rate_numubar = dR_dEr_numubar(Ers, M, 22, 18, N_POT*Norm_Ar_COHERENT_ton)
rate_nue = (N_POT*Norm_Ar_COHERENT_ton)*dR_dEr_nue(Ers, M, 22, 18, 1)
#plt.plot(Ers*1000, rates, c="black", label="Total")
plt.plot(Ers*1000, rate_nue, label=r"$\nu_{e}$")
plt.plot(Ers*1000, rate_numu, label=r"$\nu_{\mu}$")
plt.plot(Ers*1000, rate_numubar, label=r"$\bar{\nu}_{\mu}$")
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.scatter([], [], c="black", label="Detector Mass "+str(mdet)+" kg")
plt.scatter([], [], c="black", label="Baseline "+str(L)+" m")
#plt.yscale("log")
plt.xlim([1, 140])
plt.ylim([0, np.max(rate_numu)])
plt.ylabel("Events/year", fontsize=14)
plt.legend(loc="upper right")
plt.show()

In [ ]:
print(M)
print(N_POT)
print(Norm_Ar_COHERENT_ton)
print(Norm_Ar_COHERENT_ton*N_POT)

In [ ]:
# Try to find R(Er) by integrating

def R_numubar(Er):
    dEr = Er[1] - Er[0]
    rs = []
    for E in Er:
        r = quad(dR_dEr_numubar, E, E+dEr, args=(M, 22, 18, N_POT*Norm_Ar_COHERENT_ton))[0]
        rs.append(r)
    return np.array(rs)

def R_nue(Er):
    dEr = Er[1] - Er[0]
    rs = []
    for E in Er:
        r = quad(dR_dEr_nue, E, E+dEr, args=(M, 22, 18, N_POT*Norm_Ar_COHERENT_ton))[0]
        rs.append(r)
    return np.array(rs)

def R_numu(Er):
    dEr = Er[1] - Er[0]
    rs = []
    for E in Er:
        r = quad(dR_dEr_numu, E, E+dEr, args=(M, 22, 18, N_POT*Norm_Ar_COHERENT_ton))[0]
        rs.append(r)
    return np.array(rs)

Ers1 = np.arange(0.001, 0.14, 0.001)
plt.plot(Ers1*1000, R_numubar(Ers1), label=r"$\bar{\nu}_{\mu}$")
plt.plot(Ers1*1000, R_nue(Ers1), label=r"$\nu_{e}$")
plt.plot(Ers1*1000, R_numu(Ers1), label=r"$\nu_{\mu}$")
plt.ylabel("Events/year/keV", fontsize=14)
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.legend(fontsize=14)
plt.xlim([1, 140])
plt.ylim([0, 350])
plt.show()

In [ ]:
# Get the number of predicted events

N_numubar = quad(dR_dEr_numubar, 0, 0.14, args=(M, 22, 18, N_POT*Norm_Ar_COHERENT_ton))[0]
N_nue = quad(dR_dEr_nue, 0, 0.14, args=(M, 22, 18, N_POT*Norm_Ar_COHERENT_ton))[0]
print(N_numubar + N_nue)

# Argon 

In [ ]:
print(gV(22, 18)**2)
print(gV(18, 18)**2)

In [ ]:
print((22**2)/4.)

In [ ]:
x = 22 - ((1 - 4*0.231)*18)
print((x**2)/4)

In [ ]:
print(N_POT)

In [ ]:
print(N_POT*Norm_Ar_COHERENT_ton)

In [ ]:
r1 = (10**7)*(3600*24*365)*(1/(2.1*10**23))*(100**2)*(4*math.pi*(20**2))
print(r1)

# Angular Spectrum

In [ ]:
from matplotlib.colors import LogNorm

def epsilon(Er, costhetar, M):
    Enu_min = get_nu_min_energy(Er, M)
    return 1.0/((costhetar/Enu_min) - (1.0/M))

def dR_dEr_dcos_muon(Er, costhetar, M, N, Z, Norm):
    Enu_min = get_nu_min_energy(Er, M)
    e = epsilon(Er, costhetar, M)
    sig = dSigma_dEr(Er, e, M, N, Z)
    Fsquared = FH(Er, M, Rmin)**2
    a = (e**2)/Enu_min
    f_nue = flux_nue(e)
    f_numubar = flux_numubar(e)
    return (Norm/(2*math.pi))*a*Fsquared*sig*(f_nue + f_numubar)

Ers = np.linspace(0.001, 0.16, 1000)
costhetas = np.linspace(0.01, 1, 1000)
#rate_muon = dR_dEr_dcos_muon(Ers, costhetas, M, 22, 18, N_POT*Norm_Ar_COHERENT_ton)

X, Y = np.meshgrid(Ers, costhetas)     # Create a grid
#Z = rate_muon  # Define a function over the grid

Z = dR_dEr_dcos_muon(X, Y, M, 22, 18, N_POT*Norm_Ar_COHERENT_ton)

# Create a contour plot
plt.figure(figsize=(8, 6))

# Contour lines
#contour = plt.contour(X*1000, Y, Z, levels=100, cmap="viridis", norm=LogNorm())  # 20 levels
#plt.clabel(contour, inline=True, fontsize=8)               # Label the contours

# Filled contour
contour = plt.contourf(X*1000, Y, Z, levels=100, 
                       cmap="viridis", alpha=0.7, norm=LogNorm())  # Filled contours

# Add a colorbar
#plt.colorbar(label=r"$\frac{dN}{dE_{r}d\Omega_{r}}$", labelsize=14)
colorbar = plt.colorbar(contour)
colorbar.set_label(r"$\frac{dN}{dE_{r}d\Omega_{r}}$"+" Events/year/keV/sr", fontsize=14)

plt.scatter([], [], c="black", label="Detector Mass: "+str(mdet)+ " kg")
plt.scatter([], [], c="black", label="Baseline: "+str(L)+ " m")

#plt.xlim([0, 150])
plt.ylim([0, 1.1])
# Add labels
#plt.title("Contour Plot Example")
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.ylabel("cos"+r"$\theta_{r}$", fontsize=14)
plt.xscale("log")
# Show the plot
plt.xlim([0, 200])
plt.legend(loc="lower right")
plt.title(r"$\nu_{e} + \bar{\nu}_{\mu}$"+" Flux Contribution", fontsize=20)
plt.show()

In [ ]:
from scipy.integrate import simps
from scipy.integrate import dblquad

def R_double_muon(Er, costhetar, M, N, Z, Norm, dE, dcos):
    #dEr = Er[1]-Er[0]
    #dcosthetar = costhetar[1] - costhetar[0]
    
    def inner_integral(costhetar, M, N, Z, Norm):
        return quad(dR_dEr_dcos_muon, costhetar, costhetar+dcos, args=(costhetar, M, N, Z, Norm))[0]


    I = quad(inner_integral, Er, Er+dE, args=(costhetar, M, N, Z, Norm))[0]
                
    return I


"""
Ers = np.arange(0.001, 0.16, 0.001)
costhetas = np.arange(1/(4*math.pi), 1, 1/(4*math.pi))
#rate_muon = dR_dEr_dcos_muon(Ers, costhetas, M, 22, 18, N_POT*Norm_Ar_COHERENT_ton)

X, Y = np.meshgrid(Ers, costhetas)     # Create a grid

#Z = dR_dEr_dcos_muon(X, Y, M, 22, 18, N_POT*Norm_Ar_COHERENT_ton)



local_integrals = np.zeros_like(X)

dx = 0.001
dy = 1/(4*math.pi)

# Loop over each point in the grid
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        x_center = X[i, j]
        y_center = Y[i, j]
        
        # Perform local integration around (x_center, y_center)
        integral, _ = dblquad(
            dR_dEr_dcos_muon,
            y_center - (dy/2), y_center + (dy/2),   # y-range
            x_center - (dx/2),       # x lower bound
            x_center + (dx/2),        # x upper bound
            args=(M, 22, 18, N_POT*Norm_Ar_COHERENT_ton)
        )
        local_integrals[i, j] = integral



# Create a contour plot
plt.figure(figsize=(8, 6))

# Filled contour
contour = plt.contourf(X*1000, Y, local_integrals, levels=100, 
                       cmap="viridis", alpha=0.7, norm=LogNorm())  # Filled contours

# Add a colorbar
#plt.colorbar(label=r"$\frac{dN}{dE_{r}d\Omega_{r}}$", labelsize=14)
colorbar = plt.colorbar(contour)
colorbar.set_label(r"$\frac{dN}{dE_{r}d\Omega_{r}}$"+" Events/year/keV/sr", fontsize=14)

plt.scatter([], [], c="black", label="Detector Mass: "+str(mdet)+ " kg")
plt.scatter([], [], c="black", label="Baseline: "+str(L)+ " m")

#plt.xlim([0, 150])
plt.ylim([0, 1.1])
# Add labels
#plt.title("Contour Plot Example")
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.ylabel("cos"+r"$\theta_{r}$", fontsize=14)
plt.xscale("log")
# Show the plot
plt.xlim([0.5, 200])
plt.legend(loc="lower right")
plt.title(r"$\nu_{e} + \bar{\nu}_{\mu}$"+" Flux Contribution", fontsize=20)
plt.show()
"""

In [ ]:
from scipy.ndimage import gaussian_filter

dx = 0.0001
dy = 0.005

Ers = np.arange(dx, 0.2+dx, dx)
costhetas = np.arange(dy, 1+dy, dy)

#print(Ers)
#print(costhetas)

#rate_muon = dR_dEr_dcos_muon(Ers, costhetas, M, 22, 18, N_POT*Norm_Ar_COHERENT_ton)

X, Y = np.meshgrid(Ers, costhetas)     # Create a grid

Z = dR_dEr_dcos_muon(X, Y, M, 22, 18, N_POT*Norm_Ar_COHERENT_ton)


integral_approx = gaussian_filter(Z, sigma=(dx/2, dy/2))

# Create a contour plot
plt.figure(figsize=(8, 6))

# Filled contour
contour = plt.contourf(X*1000, Y, integral_approx, levels=1000, 
                       cmap="viridis", alpha=0.7, norm=LogNorm())  # Filled contours

# Add a colorbar
#plt.colorbar(label=r"$\frac{dN}{dE_{r}d\Omega_{r}}$", labelsize=14)
colorbar = plt.colorbar(contour)
colorbar.set_label(r"$\frac{dN}{dE_{r}d\Omega_{r}}$"+" Events/year/keV/sr", fontsize=14)

plt.scatter([], [], c="black", label="Detector Mass: "+str(mdet)+ " kg")
plt.scatter([], [], c="black", label="Baseline: "+str(L)+ " m")
#plt.plot([1, 200], [1/(4*math.pi), 1/(4*math.pi)], c="r")
#plt.xlim([0, 150])
plt.ylim([0, 1.1])
# Add labels
#plt.title("Contour Plot Example")
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.ylabel("cos"+r"$\theta_{r}$", fontsize=14)
plt.xscale("log")
# Show the plot
plt.xlim([1, 200])
plt.legend(loc="lower right")
plt.title(r"$\nu_{e} + \bar{\nu}_{\mu}$"+" Flux Contribution", fontsize=20)
plt.show()

In [ ]:
print(np.sum(integral_approx[(integral_approx > 0)]))

In [ ]:
plt.figure(figsize=(8, 6))
heatmap = plt.imshow(
    integral_approx, 
    extent=(Ers.min()*1000, Ers.max()*1000, costhetas.min(), costhetas.max()),  # Set the axes to match the meshgrid
    origin='lower',  # Place the (0, 0) in the lower-left
    aspect='auto',   # Adjust aspect ratio
    cmap='viridis',   # Color map
    norm=LogNorm() 
)
plt.colorbar(heatmap, label='Local Integral Value')  # Add a colorbar
plt.xlabel('Nuclear Recoil Energy [keV]', fontsize=14)
plt.ylabel("cos"+r"$\theta_{r}$", fontsize=14)
plt.title(r"$\nu_{e} + \bar{\nu}_{\mu}$"+" Flux Contribution", fontsize=14)
plt.xscale("log")
plt.xlim([0.1, 200])
plt.ylim([0.0, 1.1])
plt.show()